In [2]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, rigidity_modulus
from concrete_design.steel_reinforcement import rebar_df as rebar
from masonry_design.material_properties import NW_CMU, cmu_elastic_modulus
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Column Attachment to Plate

## Moment
### PJP Weld to Plate

#### Tension Force

In [9]:
Mu = 185.2 #kip-ft

In [17]:
beam = "W16x57"
d = aisc.loc[beam, "d"]
tf = aisc.loc[beam, "tf"]
bf = aisc.loc[beam, "bf"]
beam_ecc = d - tf
f"tf = {tf}; beam eccentricity = {round(beam_ecc, 2)} in"

'tf = 0.715; beam eccentricity = 15.68 in'

In [14]:
Tu = Mu / (beam_ecc/12)
f"Tu from flange = {round(Tu, 2)} kips"

'Tu from flange = 141.69 kips'

#### Weld Design

In [19]:
phi = 0.80
F_EXX = 70 #ksi
Fnw = 0.60*F_EXX
throat = 0.625 #in
Awe = bf * throat

Rn = Fnw * Awe #kips
f"phiRn = {round(phi*Rn, 2)} kips"

'phiRn = 149.52 kips'

--> Use full CJP

#### Base Metal Check

In [20]:
phi = 0.75
FnBM = 36 #ksi
t_BM = 2 #in
A_BM = bf * t_BM #in^2

Rn = FnBM * A_BM
f"phiRn = {round(phi*Rn, 2)} kips"

'phiRn = 384.48 kips'

# Anchor Bolt Embedment Design

## Strut-N-TieDemands

In [ ]:
anchor_embed = 36 #in
anchor_ecc = 22 #in
caisson_dia = 42 #in
conc_cover = 3 #in

In [ ]:
Tu = Mu / (ecc/12) #kips
f"Tu = {round(Tu, 2)} kips"

In [ ]:
angle = arctan(anchor_embed / (anchor_ecc/2 + (caisson_dia-2*conc_cover)/2))
f"angle = {round(rad2deg(angle), 2)} degrees"

In [ ]:
T1 = T3 = Tu

In [ ]:
S1 = T1 / sin(angle) #kips
f"S1 = {round(S1, 2)} kips"

In [ ]:
T2 = T1 / tan(angle)
f"T2 = T4 = {round(T2, 2)} kips"

## Strut-N-Tie Design

### S1 Check

In [ ]:
phi = 0.75
fpr_c = 3 #ksi
fy = 60 #ksi

In [ ]:
Acs = 9 * 28 #in^2 (conservative estimate)

beta_c = 1.0
beta_s = 0.4
fce = 0.85 * beta_c * beta_s * fpr_c

Fns1 = fce * Acs
f"phi * Fns1 = {round(phi * Fns1, 2)} kips"

### N1 Check

In [ ]:
Anz = Acs #in^2 (conservative estimate)

beta_n = 0.6
fce = 0.85 * beta_c * beta_n * fpr_c

Fnn1 = fce * Anz #kips
f"phi * Fnn1 = {round(phi * Fnn1, 2)} kips"

### T1 Design

In [ ]:
bar_size = "#5"
Ats = 6 * rebar.loc[bar_size, "Area"] #in^2

Fnt2 = Ats * fy

f"phi * Fnt2 = {round(phi * Fnt2, 2)} kips"

--> Design is adequate by inspection without further checks

# Caisson Reinforcement

In [ ]:
fpr_c = 3000 #psi
fy = 60000 #psi

### Ties

In [ ]:
fyt = fy
bw = sqrt(pi*caisson_dia**2 / 4) #in
s = 12 #in
Av_min = max(0.75 * sqrt(fpr_c) * bw*s/fyt, 50 * bw*s/fyt)
f"Av_min = {round(Av_min, 3)} in^2 at {s} in O.C."